In [4]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
L=LaurentSeriesRing(QQ,'x', default_prec=300)

def polynomialdivision(dividend, divisor):
    q,r = dividend.maxima_methods().divide(divisor)
    return (q,r)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return L(xjNew(n,m)/x)

def H(n,m): # = Hlambda
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.
    # bad code

def Hstrike(n,m): 
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def quotientFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[0]
def remainderFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[1]
def factor(polynomial):
    fp=polynomial.factor()
    return fp
def numberOfFactors(polynomial):
    fp=polynomial.factor()
    return len(fp)
def factorK(polynomial,k):
    fp=factor(h)
    return fp[k-1]
def factorList(polynomial):
    answer=[]
    nf=numberOfFactors(polynomial)
    for k in [1..nf]:
        answer=answer+[factorK(polynomial,k)]
    return answer
def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel
def polynomialFactorBaseList(polynomial):
    fbl=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        fbl=fbl+[base]
    return fbl

print("ok1")

ok1


In [2]:
poly=Hstrike(5,3);
print(poly)

30240*x^5 + 17520*x^4 + 6720*x^3 + 2160*x^2 + 240*x + 1


In [6]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [3]:
diffs=[]
poly=Hstrike(100,3)
plist=poly.list()
for k in [1..poly.degree()]:
    diffs=diffs+[240*sigma(k,3)-plist[k]]
print(diffs)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
data=[]
no=[]
import time
startall=time.time()
for m in [3..603]:
    poly=Hstrike(200,m)
    pd=poly.degree()
    if pd!=200:no=no+[m]
    data=data+[[m,poly]]
    finish=time.time()
    print([m,pd,(finish-startall)/60])
import pickle
wfile = open('/Users/barrybrent/7sept22no1.txt','wb') # Hstrike series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()
print("wrong pd:")
print(no)

[3, 200, 0.2525535702705383]
[4, 200, 0.4232491691907247]
[5, 200, 0.7651151537895202]
[6, 199, 0.9239761829376221]
[7, 200, 1.3032394170761108]
[8, 200, 1.5966440637906392]
[9, 200, 2.0151574691136678]
[10, 200, 2.278028432528178]
[11, 200, 2.694209416707357]
[12, 200, 3.055611085891724]
[13, 200, 3.5017030000686646]
[14, 200, 3.7923790136973063]
[15, 200, 4.2892352859179175]
[16, 200, 4.64275320370992]
[17, 200, 5.135199069976807]
[18, 200, 5.460302885373434]
[19, 200, 5.937328469753266]
[20, 200, 6.388353431224823]
[21, 200, 6.972115015983581]
[22, 200, 7.31541250149409]
[23, 200, 7.825251336892446]
[24, 200, 8.2695081671079]
[25, 200, 8.796484132607779]
[26, 200, 9.152969781557719]
[27, 200, 9.674064215024313]
[28, 200, 10.116232069333394]
[29, 200, 10.613437847296398]
[30, 200, 11.001011248429617]
[31, 200, 11.497160617510477]
[32, 200, 11.906918485959372]
[33, 200, 12.47677076657613]
[34, 200, 12.84423198302587]
[35, 200, 13.427868334452311]
[36, 200, 13.918396083513896]
[37, 200

[276, 200, 158.11668946743012]
[277, 200, 158.8010812997818]
[278, 200, 159.33170646429062]
[279, 200, 160.0650177637736]
[280, 200, 160.75029118061065]
[281, 200, 161.43323518037795]
[282, 200, 162.01587260166804]
[283, 200, 162.7149342338244]
[284, 200, 163.3450622677803]
[285, 200, 164.12759600083032]
[286, 200, 164.6772883494695]
[287, 200, 165.42387623786925]
[288, 200, 166.06175371805827]
[289, 200, 166.80395058393478]
[290, 200, 167.37215330203375]
[291, 200, 168.10887528260548]
[292, 200, 168.72199523448944]
[293, 200, 169.42280608415604]
[294, 200, 170.01757063468298]
[295, 200, 170.75114489793776]
[296, 200, 171.37464908361434]
[297, 200, 172.11374398072562]
[298, 200, 172.6364677508672]
[299, 200, 173.37240910132726]
[300, 200, 174.0598889350891]
[301, 200, 174.79933583339056]
[302, 200, 175.3171667655309]
[303, 200, 176.06272211472194]
[304, 200, 176.68769359985987]
[305, 200, 177.4321106513341]
[306, 200, 178.01130774815877]
[307, 200, 178.7255464831988]
[308, 200, 179.407

[545, 200, 347.0457612037659]
[546, 200, 347.7054263154666]
[547, 200, 348.44405648708346]
[548, 200, 349.1510335803032]
[549, 200, 349.97719226678214]
[550, 200, 350.64119100173315]
[551, 200, 351.42974471648535]
[552, 200, 352.1642887314161]
[553, 200, 352.9537914474805]
[554, 200, 353.5479912519455]
[555, 200, 354.40907824834187]
[556, 200, 355.0858145356178]
[557, 200, 355.873602883021]
[558, 200, 356.5104373852412]
[559, 200, 357.3141678015391]
[560, 200, 358.05262682040535]
[561, 200, 358.90170943339666]
[562, 200, 359.5208210349083]
[563, 200, 360.2970712184906]
[564, 200, 361.0540663162867]
[565, 200, 361.8628688335419]
[566, 200, 362.47899251381557]
[567, 200, 363.3482928832372]
[568, 200, 364.0447824160258]
[569, 200, 364.78914231856663]
[570, 200, 365.4673532009125]
[571, 200, 366.22409925063454]
[572, 200, 366.93873011668524]
[573, 200, 367.74732370376586]
[574, 200, 368.3749850988388]
[575, 200, 369.23052080074945]
[576, 200, 369.95562485059105]
[577, 200, 370.737082453568

In [10]:
import pickle
rfile = open('/Users/barrybrent/10aug21no1.txt','r') # Hstrike series 
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

401

In [11]:
import pickle
rfile = open('/Users/barrybrent/10aug21no1.txt','r') # Hstrike series 
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import pickle
import time
rfile = open('/Users/barrybrent/10aug21no2.txt','w') #Hstrike polynomials
for n in [0..100]:
    start=time.time()
    data=[]
    for k in [0..400]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
        rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]]
    finish=time.time()
    print([n,rl.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t) # bad code bc I mistakenly named the file rfile not wfile.
wfile.close()

[0, 0, 31.138643980026245]
[1, 2, 30.54431700706482]
[2, 5, 30.420740842819214]
[3, 8, 30.174880981445312]
[4, 11, 31.228797912597656]
[5, 14, 31.464418172836304]
[6, 17, 31.819730043411255]
[7, 20, 31.368056058883667]
[8, 23, 31.93084406852722]
[9, 26, 32.07727599143982]
[10, 29, 32.37785816192627]
[11, 32, 33.66185188293457]
[12, 35, 33.76943612098694]
[13, 38, 34.06246900558472]
[14, 41, 34.0097758769989]
[15, 44, 34.84329891204834]
[16, 47, 36.66833305358887]
[17, 50, 36.66372299194336]
[18, 53, 37.59294080734253]
[19, 56, 36.630422830581665]
[20, 59, 36.80771589279175]
[21, 62, 37.4052939414978]
[22, 65, 38.346081018447876]
[23, 68, 39.32688903808594]
[24, 71, 39.994043827056885]
[25, 74, 41.088836908340454]
[26, 77, 41.61211085319519]
[27, 80, 43.6242561340332]
[28, 83, 44.38481593132019]
[29, 86, 45.96862292289734]
[30, 89, 46.533612966537476]
[31, 92, 47.99377202987671]
[32, 95, 48.44652605056763]
[33, 98, 49.47912383079529]
[34, 101, 50.7105028629303]
[35, 104, 51.001181125640

ValueError: I/O operation on closed file

In [12]:
t = pickle.dumps(str(polydata)) # here I write above data to file.
rfile.write(t)
rfile.close()

In [9]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..3]:
    print("==================================================")
    print("index: ",s[k][0])
    print("poly:")
    poly=D(s[k][1])
    deg=poly.degree()
    print(poly)
    print("degree: ",deg)

index:  0
poly:
1
degree:  0
index:  1
poly:
16*x^2 + 32*x
degree:  2
index:  2
poly:
-16*x^5 + 96*x^4 + 64*x^3 - 384*x^2
degree:  5
index:  3
poly:
64/3*x^8 - 640/3*x^7 + 9728/9*x^6 - 11264/9*x^5 - 41984/9*x^4 + 75776/9*x^3
degree:  8


In [11]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
lns=len(s)
data=[]
for k in [0..lns-1]:
    poly=D(s[k][1])
    deg=poly.degree()
    data=data+[deg]
print(data)
    

[0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 134, 137, 140, 143, 146, 149, 152, 155, 158, 161, 164, 167, 170, 173, 176, 179, 182, 185, 188, 191, 194, 197, 200, 203, 206, 209, 212, 215, 218, 221, 224, 227, 230, 233, 236, 239, 242, 245, 248, 251, 254, 257, 260, 263, 266, 269, 272, 275, 278, 281, 284, 287, 290, 293, 296, 299]


In [13]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    print("=====================================================")
    n=s[k][0]
    print("index: ",n)
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print(factor(f))
    pel=polynomialExponentList(f)
    nf=numberOfFactors(f)
    print([nf,nfc,pel])

('index: ', 1)
(16) * x * (x + 2)
[2, 16, [1, 1]]
('index: ', 2)
(-16) * (x - 6) * (x - 2) * (x + 2) * x^2
[4, -16, [1, 1, 1, 2]]
('index: ', 3)
(64/3) * (x - 2) * (x + 2) * x^3 * (x^3 - 10*x^2 + 164/3*x - 296/3)
[4, 64/3, [1, 1, 3, 1]]
('index: ', 4)
(-16) * (x - 2) * (x + 2) * x^4 * (x^5 - 82/3*x^4 + 6152/27*x^3 - 28976/27*x^2 + 82480/27*x - 99616/27)
[4, -16, [1, 1, 4, 1]]
('index: ', 5)
(96/5) * (x - 6) * (x - 2) * (x + 2) * x^5 * (x^6 - 208/9*x^5 + 110276/405*x^4 - 221824/135*x^3 + 2422256/405*x^2 - 1938176/135*x + 6657728/405)
[5, 96/5, [1, 1, 1, 5, 1]]


In [14]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

101

In [15]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..100]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    data=data+[[n,nfc]]
print(data)

[[1, 16], [2, -16], [3, 64/3], [4, -16], [5, 96/5], [6, -64/3], [7, 128/7], [8, -16], [9, 208/9], [10, -96/5], [11, 192/11], [12, -64/3], [13, 224/13], [14, -128/7], [15, 128/5], [16, -16], [17, 288/17], [18, -208/9], [19, 320/19], [20, -96/5], [21, 512/21], [22, -192/11], [23, 384/23], [24, -64/3], [25, 496/25], [26, -224/13], [27, 640/27], [28, -128/7], [29, 480/29], [30, -128/5], [31, 512/31], [32, -16], [33, 256/11], [34, -288/17], [35, 768/35], [36, -208/9], [37, 608/37], [38, -320/19], [39, 896/39], [40, -96/5], [41, 672/41], [42, -512/21], [43, 704/43], [44, -192/11], [45, 416/15], [46, -384/23], [47, 768/47], [48, -64/3], [49, 912/49], [50, -496/25], [51, 384/17], [52, -224/13], [53, 864/53], [54, -640/27], [55, 1152/55], [56, -128/7], [57, 1280/57], [58, -480/29], [59, 960/59], [60, -128/5], [61, 992/61], [62, -512/31], [63, 1664/63], [64, -16], [65, 1344/65], [66, -256/11], [67, 1088/67], [68, -288/17], [69, 512/23], [70, -768/35], [71, 1152/71], [72, -208/9], [73, 1184/73], 

In [5]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [1..10]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print([n,nfc])


[1, 16]
[2, -16]
[3, 64/3]
[4, -16]
[5, 96/5]
[6, -64/3]
[7, 128/7]
[8, -16]
[9, 208/9]
[10, -96/5]


In [18]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r')#Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if nfc in ZZ:data=data+[n]
print(data)

[1, 2, 4, 8, 16, 32, 64]


In [19]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if 2*nfc in ZZ:data=data+[n]
print(data)

[1, 2, 4, 8, 16, 32, 64]


In [6]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if 3*nfc in ZZ:data=data+[n]
print(data)

[1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96]


In [21]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    num=nfc.numerator()
    data=data+[num]
print(data)

[16, -16, 64, -16, 96, -64, 128, -16, 208, -96, 192, -64, 224, -128, 128, -16, 288, -208, 320, -96, 512, -192, 384, -64, 496, -224, 640, -128, 480, -128, 512, -16, 256, -288, 768, -208, 608, -320, 896, -96, 672, -512, 704, -192, 416, -384, 768, -64, 912, -496, 384, -224, 864, -640, 1152, -128, 1280, -480, 960, -128, 992, -512, 1664, -16, 1344, -256, 1088, -288, 512, -768, 1152, -208, 1184, -608, 1984, -320, 1536, -896, 1280, -96, 1936, -672, 1344, -512, 1728, -704, 640, -192, 1440, -416, 256, -384, 2048, -768, 384, -64, 1568, -912, 832, -496]


In [23]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    num=nfc.numerator()
    data=data+[num/16]
print(data) A098986, A336113

[1, -1, 4, -1, 6, -4, 8, -1, 13, -6, 12, -4, 14, -8, 8, -1, 18, -13, 20, -6, 32, -12, 24, -4, 31, -14, 40, -8, 30, -8, 32, -1, 16, -18, 48, -13, 38, -20, 56, -6, 42, -32, 44, -12, 26, -24, 48, -4, 57, -31, 24, -14, 54, -40, 72, -8, 80, -30, 60, -8, 62, -32, 104, -1, 84, -16, 68, -18, 32, -48, 72, -13, 74, -38, 124, -20, 96, -56, 80, -6, 121, -42, 84, -32, 108, -44, 40, -12, 90, -26, 16, -24, 128, -48, 24, -4, 98, -57, 52, -31]


In [22]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    den=nfc.denominator()
    data=data+[den]
print(data) # OEIS A098985

[1, 1, 3, 1, 5, 3, 7, 1, 9, 5, 11, 3, 13, 7, 5, 1, 17, 9, 19, 5, 21, 11, 23, 3, 25, 13, 27, 7, 29, 5, 31, 1, 11, 17, 35, 9, 37, 19, 39, 5, 41, 21, 43, 11, 15, 23, 47, 3, 49, 25, 17, 13, 53, 27, 55, 7, 57, 29, 59, 5, 61, 31, 63, 1, 65, 11, 67, 17, 23, 35, 71, 9, 73, 37, 75, 19, 77, 39, 79, 5, 81, 41, 83, 21, 85, 43, 29, 11, 89, 15, 13, 23, 93, 47, 19, 3, 97, 49, 33, 25]


In [24]:
print divisors(12)

[1, 2, 3, 4, 6, 12]


In [33]:
def divisorsum(n):
    dvs=divisors(n)
    sm=0
    for k in [0..len(dvs)-1]:
        d=dvs[k]
        if mod(d,2)==1:sm=sm+1/dvs[k]
    return sm
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print (nfc,(-1)^(n+1)*16*divisorsum(n))

        
        

(16, 16)
(-16, -16)
(64/3, 64/3)
(-16, -16)
(96/5, 96/5)


In [8]:
def divisorsum(n):
    dvs=divisors(n)
    sm=0
    for k in [0..len(dvs)-1]:
        d=dvs[k]
        if mod(d,2)==1:sm=sm+1/dvs[k]
    return sm
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    diff=nfc-(-1)^(n+1)*16*divisorsum(n)
    data=data+[diff]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','rb') #Hstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..10]:
    n=s[k][0]
    if n in Q:
        print("=====================================================")
        print("index: ",n)
        f = D(s[k][1])
        nfc=numericalfactor(f)
        print(factor(f))
        pel=polynomialExponentList(f)
        nf=numberOfFactors(f)
        print([nf,nfc,pel])

index:  2
(-16) * (x - 6) * (x - 2) * (x + 2) * x^2
[4, -16, [1, 1, 1, 2]]
index:  5
(96/5) * (x - 6) * (x - 2) * (x + 2) * x^5 * (x^6 - 208/9*x^5 + 110276/405*x^4 - 221824/135*x^3 + 2422256/405*x^2 - 1938176/135*x + 6657728/405)
[5, 96/5, [1, 1, 1, 5, 1]]
index:  6
(-64/3) * (x - 6) * (x - 2) * (x + 2) * x^6 * (x^8 - 436/15*x^7 + 520216/1125*x^6 - 4782512/1125*x^5 + 87678976/3375*x^4 - 71574208/675*x^3 + 109029248/375*x^2 - 1824434944/3375*x + 1743597824/3375)
[5, -64/3, [1, 1, 1, 6, 1]]
index:  8
(-16) * (x - 6) * (x - 2) * (x + 2) * x^8 * (x^12 - 6848/105*x^11 + 1774828808/1157625*x^10 - 82609634816/3472875*x^9 + 7866232062544/31255875*x^8 - 20506551003136/10418625*x^7 + 73525770854656/6251175*x^6 - 589961658368/11025*x^5 + 1949052254920448/10418625*x^4 - 574622563352576/1157625*x^3 + 29980732221446144/31255875*x^2 - 13536157469376512/10418625*x + 30787110303281152/31255875)
[5, -16, [1, 1, 1, 8, 1]]
index:  10
(-96/5) * (x - 6) * (x - 2) * (x + 2) * x^10 * (x^16 - 68312/945*x^15 + 